In [2]:
from plotly.subplots import make_subplots
from analysing_functions import *
from IIRfilter import *
# IF you are using Pycharm
import plotly.io as pio
from scipy import fftpack, signal, stats

target = 4
env = "W"
block = 2
subject = 303
## Bring the result into pandas dataframe
holo, imu, eye = bring_data(target, env, block, subject)
## Get the delayed time between hololens - laptop
shift, corr, shift_time = synchronise_timestamp(imu, holo, show_plot=False)

## filter out the low-confidene eye data
# eye = eye[eye.confidence > 0.8]

## match the delayed timestamp into hololens' timestamp
eye.timestamp = eye.timestamp - shift_time
# eye.norm_y = eye.norm_y - eye.norm_y.mean()
## For easy manipulation/check, make all dataframes into 120Hz, and make same timestamp by interpolation (1-dimensional)
new_holo, new_imu, new_eye = interpolated_dataframes(holo, imu, eye)

In [11]:
def asSpherical(xyz: list):
    # takes list xyz (single coord)
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    r = math.sqrt(x * x + y * y + z * z)
    theta = math.acos(z / r) * 180 / math.pi  # to degrees
    phi = math.atan2(y, x) * 180 / math.pi
    return [r, theta, phi]

def calculate_max_angle(fromX,fromY,fromZ,toX,toY,toZ,r):
    length = np.sqrt((toX-fromX)**2 + (toY-fromY)**2 + (toZ-fromZ)**2)
    angle_in_rad=np.arcsin(r/length)

    return np.rad2deg(angle_in_rad)

def calculate_current_angle(headX,headY,headZ,fromX,fromY,fromZ,toX,toY,toZ):
    import vg
    vector1 = np.array([headX,headY,headZ])
    vector2 = np.array([toX-fromX,toY-fromY,toZ-fromZ])
    return vg.angle(vector1,vector2)

def add_columns(holo):
    rs = []
    thetas = []
    phis = []
    for index, row in holo.iterrows():
        x = row['target_position_x'] - row['head_position_x']
        y = row['target_position_y'] - row['head_position_y']
        z = row['target_position_z'] - row['head_position_z']
        [r, theta, phi] = asSpherical([x, z, y])
        rs.append(r)
        thetas.append(90 - theta)
        phis.append(90 - phi)

    holo['R'] = rs
    holo['Theta'] = thetas
    holo['Phi'] = phis
    holo['TargetVertical'] = holo.head_rotation_x + holo.Theta
    holo['TargetHorizontal'] = holo.head_rotation_y - holo.Phi
    for col in ['head_forward_x', 'head_forward_y', 'head_forward_z']:
        holo[str(col) + '_next'] = holo[col].shift(1)
    holo['time_interval'] = holo.timestamp.diff()
    holo['angle_speed'] = holo.apply(lambda x: angle_velocity(x.head_forward_x, x.head_forward_y, x.head_forward_z,
                                                                        x.head_forward_x_next, x.head_forward_y_next,
                                                                        x.head_forward_z_next,
                                                                        x.time_interval), axis=1)
    return holo

def angle_velocity(x1, y1, z1, x2, y2, z2, _time):
    import vg
    # if type(_head_forward2) is not dict: return None
    vector1 = np.array([x1, y1, z1])
    vector2 = np.array([x2, y2, z2])
    return vg.angle(vector1, vector2) / _time

subjects = subjects = range(301, 317)
poss = ['W']
envs = ['U','W']
targets=range(8)
# blocks=range(5)
blocks = [1,2,3,4]
output=pd.DataFrame()
for subject in subjects:
    for target, env, pos, block in itertools.product(targets, envs, poss, blocks):
        holo, imu, eye = bring_data(target, env, block, subject)
        current_info = [target, env, pos, block]
        print(subject,current_info)
        ## Get the delayed time between hololens - laptop
        shift, corr, shift_time = synchronise_timestamp(imu, holo, show_plot=False)

        holo.timestamp = holo.timestamp - holo.timestamp[0]
        holo = add_columns(holo)
        if holo[holo.target_entered ==True].shape[0]<1:
            print('no target-in');continue

        start_index= holo[ holo.target_entered==True].index[0]
        holo_after_target_on= holo[holo.index>start_index]
        accuracy_vertical = -holo_after_target_on.Theta.mean()-holo_after_target_on.head_rotation_x.mean()
        accuracy_horizontal=holo_after_target_on.Phi.mean()-holo_after_target_on.head_rotation_y.mean()
        precision_vertical=(holo_after_target_on.TargetVertical).std()
        precision_horizontal=(holo_after_target_on.TargetHorizontal).std()
        result =dict(
            subject=subject,target=target,env=env,block=block,
            accuracy_vertical=accuracy_vertical,accuracy_horizontal=accuracy_horizontal,
            precision_vertical=precision_vertical,precision_horizontal=precision_horizontal
        )
        output=output.append(result,ignore_index=True)

301 [0, 'U', 'W', 1]
301 [0, 'U', 'W', 2]
301 [0, 'U', 'W', 3]
301 [0, 'U', 'W', 4]
301 [0, 'W', 'W', 1]
301 [0, 'W', 'W', 2]
301 [0, 'W', 'W', 3]
301 [0, 'W', 'W', 4]
301 [1, 'U', 'W', 1]
301 [1, 'U', 'W', 2]
301 [1, 'U', 'W', 3]
301 [1, 'U', 'W', 4]
301 [1, 'W', 'W', 1]
301 [1, 'W', 'W', 2]
301 [1, 'W', 'W', 3]
301 [1, 'W', 'W', 4]
301 [2, 'U', 'W', 1]
301 [2, 'U', 'W', 2]
301 [2, 'U', 'W', 3]
301 [2, 'U', 'W', 4]
301 [2, 'W', 'W', 1]
301 [2, 'W', 'W', 2]
301 [2, 'W', 'W', 3]
301 [2, 'W', 'W', 4]
301 [3, 'U', 'W', 1]
301 [3, 'U', 'W', 2]
301 [3, 'U', 'W', 3]
301 [3, 'U', 'W', 4]
301 [3, 'W', 'W', 1]
301 [3, 'W', 'W', 2]
301 [3, 'W', 'W', 3]
301 [3, 'W', 'W', 4]
301 [4, 'U', 'W', 1]
301 [4, 'U', 'W', 2]
301 [4, 'U', 'W', 3]
301 [4, 'U', 'W', 4]
301 [4, 'W', 'W', 1]
301 [4, 'W', 'W', 2]
301 [4, 'W', 'W', 3]
301 [4, 'W', 'W', 4]
301 [5, 'U', 'W', 1]
301 [5, 'U', 'W', 2]
301 [5, 'U', 'W', 3]
301 [5, 'U', 'W', 4]
301 [5, 'W', 'W', 1]
301 [5, 'W', 'W', 2]
301 [5, 'W', 'W', 3]
301 [5, 'W', 

In [13]:
UI_accuracy_H = output[output.env=='U'].accuracy_horizontal.mean()
UI_accuracy_V = output[output.env=='U'].accuracy_vertical.mean()
World_accuracy_H =output[output.env=='W'].accuracy_horizontal.mean()
World_accuracy_V =output[output.env=='W'].accuracy_vertical.mean()

UI_precision_H = output[output.env=='U'].precision_horizontal.mean()
UI_precision_V = output[output.env=='U'].precision_vertical.mean()
World_precision_H =output[output.env=='W'].precision_horizontal.mean()
World_precision_V =output[output.env=='W'].precision_vertical.mean()

print('accuracy',UI_accuracy_H,UI_accuracy_V,World_accuracy_H,World_accuracy_V)
print('precision',UI_precision_H,UI_precision_V,World_precision_H,World_precision_V)


accuracy -0.005932535678883327 0.07748426154913907 -0.030753247800596444 0.10274957305906984
precision 0.9941056776623312 1.0535070841226597 0.9971665582937062 1.0838673847425893


In [14]:
# subjects = subjects = range(301, 317)
# poss = ['W']
# envs = ['U','W']
# targets=range(8)
# blocks = [1,2,3,4]
subjects=[301]
poss=['W']
envs=['U']
targets=[1]
blocks=[1]
output=pd.DataFrame()
for subject in subjects:
    for target, env, pos, block in itertools.product(targets, envs, poss, blocks):
        holo, imu, eye = bring_data(target, env, block, subject)
        current_info = [target, env, pos, block]
        print(subject,current_info)
        ## Get the delayed time between hololens - laptop
        shift, corr, shift_time = synchronise_timestamp(imu, holo, show_plot=False)

        holo.timestamp = holo.timestamp - holo.timestamp[0]
        holo = add_columns(holo)
        eye.timestamp = eye.timestamp - shift_time

        ## For easy manipulation/check, make all dataframes into 120Hz, and make same timestamp by interpolation (1-dimensional)
        new_holo, new_imu, new_eye = interpolated_dataframes(holo, imu, eye)
        for i in range(new_holo.shape[0]):

301 [1, 'U', 'W', 1]


In [17]:
print(new_holo.timestamp[2:10])
"""
 1. filter eye-data with filtfilt
 2. set window size
 3. calculate mean difference and scale for windowed-range
 4. apply shift/scale to new eye-data arrives
 5. compare accuracy/precision with raw head movement
"""

2    0.016667
3    0.025000
4    0.033333
5    0.041667
6    0.050000
7    0.058333
8    0.066667
9    0.075000
Name: timestamp, dtype: float64
